# Lab / HomeWork
## [Artificial Neural Network (ANN)]
## Feedfroward Neural Network (FFNN)

In [77]:
import numpy as np
import pandas as pd

### Data extraction and choice amount of neurons


In [78]:
data = pd.read_csv("data_ffnn_3classes.txt",header = None,sep=" ")

V_neuron= 10
W_neuron = 3

### Generation matrix Y


In [79]:
data_y = data[2]
length_y = len(data_y)
matrix_y = np.zeros((length_y,3))

def matrixY(y, length):
    for i in range(length):
        if y[i] == 0.0:
            matrix_y[i][0] = 1
        elif y[i] == 1.0:
            matrix_y[i][1] = 1
        elif y[i] == 2.0:
            matrix_y[i][2] = 1
            
matrixY(data_y, length_y)
print("dimension matrix y: " + str(matrix_y.shape))

dimension matrix y: (71, 3)


## Forward Propagation

### Input data matrix []<sub> I x N
I = Number of elements

N = Number of features

In [80]:
def input_matrix_x():
    return np.array([data[0], data[1]])
    
matrix_x = input_matrix_x().transpose()
print("dimension matrix_x: " + str(matrix_x.shape))

dimension matrix_x: (71, 2)


### Extended input data matrix []<sub> I x (N+1)

In [81]:
def x_1_matrix(x_mat):
    ones = np.ones(((np.size(x_mat, 0)),1))
    return np.append(ones, x_mat, axis=1)

matrix_x_extended = x_1_matrix(matrix_x)
print("dimension matrix_x_extended: " + str(matrix_x_extended.shape))

dimension matrix_x_extended: (71, 3)


### Extended input data matrix 2 []<sub> I x K

K = number of hidden neurons

v_matrix = []<sub> (N+1) x K

In [82]:
def x_2_matrix(x_mat_ext, v_neuron):
    return x_mat_ext.dot(v_neuron)

v_neutron_matrix = np.random.rand(np.size(matrix_x_extended, 1), V_neuron)
matrix_x_extended_2 = x_2_matrix(matrix_x_extended, v_neutron_matrix)
print("dimension matrix_x_extended_2: " + str(matrix_x_extended_2.shape))

dimension matrix_x_extended_2: (71, 10)


### Hidden F matrix []<sub> I x K


In [83]:
def f_matrix(x_mat_ext_2):
    return 1/(1 + np.exp(-x_mat_ext_2))
    
matrix_f = f_matrix(matrix_x_extended_2)
print("dimension matrix_f: " + str(matrix_f.shape))

dimension matrix_f: (71, 10)


### Extended hidden f matrix []<sub> I x (K+1)

In [84]:
def f_1_matrix(f_mat):
    ones = np.ones((np.size(f_mat,0),1))
    return np.append(ones, f_mat,axis=1)

matrix_f_extended = f_1_matrix(matrix_f)
print("dimension matrix_f_extended: " + str(matrix_f_extended.shape))

dimension matrix_f_extended: (71, 11)


### Extended hidden f matrix 2 []<sub> I x J

J = output variable dimension

w_matrix = []<sub> (K+1) x J

In [85]:
def f_2_matrix(f_mat_1, w_neuron):
    return f_mat_1.dot(w_neuron)

w_neutron_matrix = np.random.rand(V_neuron+1, W_neuron)
matrix_f_extended_2 = f_2_matrix(matrix_f_extended, w_neutron_matrix)
print("dimension matrix_f_extended_2: " + str(matrix_f_extended_2.shape))

dimension matrix_f_extended_2: (71, 3)


### G matrix []<sub> I x J


In [86]:
def g_matrix(f_mat_ext_2):
    return 1/(1 + np.exp(-f_mat_ext_2))

matrix_g = g_matrix(matrix_f_extended_2)
print("dimension matrix_g: " + str(matrix_g.shape))

dimension matrix_g: (71, 3)


### Sum of squared errors (SSE)

In [87]:
def sum_squared_errors_calculation(g_mat, y_mat):
    squares = (g_mat - y_mat ) ** 2
    return np.sum(squares)/2

sum_squared_errors = sum_squared_errors_calculation(matrix_g, matrix_y)
print("sum_squared_errors: " + str(sum_squared_errors))

sum_squared_errors: 69.96930274097349
